<a href="https://colab.research.google.com/github/ayyucedemirbas/machine_learning_algorithms/blob/master/BayesianNeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [2]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

Experiment 1: standard neural network

In [5]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/wine_quality/white/1.0.0.incompleteATA7O9/wine_quality-train.tfrecord


  0%|          | 0/4898 [00:00<?, ? examples/s]

Dataset wine_quality downloaded and prepared to /root/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


In [7]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 42ms/step - loss: 40.8024 - root_mean_squared_error: 6.3877 - val_loss: 37.5664 - val_root_mean_squared_error: 6.1291
Epoch 2/100
17/17 [==============================] - 0s 6ms/step - loss: 38.4302 - root_mean_squared_error: 6.1992 - val_loss: 35.4467 - val_root_mean_squared_error: 5.9537
Epoch 3/100
17/17 [==============================] - 0s 6ms/step - loss: 36.4107 - root_mean_squared_error: 6.0341 - val_loss: 33.5391 - val_root_mean_squared_error: 5.7913
Epoch 4/100
17/17 [==============================] - 0s 6ms/step - loss: 34.4724 - root_mean_squared_error: 5.8713 - val_loss: 31.7350 - val_root_mean_squared_error: 5.6334
Epoch 5/100
17/17 [==============================] - 0s 6ms/step - loss: 32.5810 - root_mean_squared_error: 5.7080 - val_loss: 29.9975 - val_root_mean_squared_error: 5.4770
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 30.7351 - root_mean_squared_error

In [8]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.1 - Actual: 5.0
Predicted: 6.1 - Actual: 6.0
Predicted: 4.9 - Actual: 5.0
Predicted: 6.3 - Actual: 6.0
Predicted: 6.0 - Actual: 6.0
Predicted: 5.0 - Actual: 5.0
Predicted: 6.1 - Actual: 7.0
Predicted: 5.9 - Actual: 6.0
Predicted: 6.3 - Actual: 7.0
Predicted: 6.3 - Actual: 6.0


Experiment 2: Bayesian neural network (BNN)

In [9]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [10]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 176ms/step - loss: 37.8881 - root_mean_squared_error: 6.1540 - val_loss: 37.0454 - val_root_mean_squared_error: 6.0851
Epoch 2/500
5/5 [==============================] - 0s 20ms/step - loss: 36.2797 - root_mean_squared_error: 6.0221 - val_loss: 36.3541 - val_root_mean_squared_error: 6.0281
Epoch 3/500
5/5 [==============================] - 0s 20ms/step - loss: 36.9681 - root_mean_squared_error: 6.0788 - val_loss: 37.6229 - val_root_mean_squared_error: 6.1323
Epoch 4/500
5/5 [==============================] - 0s 20ms/step - loss: 36.6180 - root_mean_squared_error: 6.0503 - val_loss: 36.5936 - val_root_mean_squared_error: 6.0481
Epoch 5/500
5/5 [==============================] - 0s 20ms/step - loss: 36.9007 - root_mean_squared_error: 6.0734 - val_loss: 33.6886 - val_root_mean_squared_error: 5.8028
Epoch 6/500
5/5 [==============================] - 0s 22ms/step - loss: 37.0520 - root_mean_squared_error: 6.08

In [12]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.11, min: 4.38, max: 5.77, range: 1.39 - Actual: 5.0
Predictions mean: 6.06, min: 5.6, max: 6.4, range: 0.8 - Actual: 6.0
Predictions mean: 5.06, min: 4.19, max: 5.81, range: 1.62 - Actual: 5.0
Predictions mean: 6.22, min: 5.83, max: 6.43, range: 0.6 - Actual: 6.0
Predictions mean: 5.86, min: 5.41, max: 6.3, range: 0.89 - Actual: 6.0
Predictions mean: 4.95, min: 4.11, max: 5.79, range: 1.68 - Actual: 5.0
Predictions mean: 6.1, min: 5.72, max: 6.36, range: 0.64 - Actual: 7.0
Predictions mean: 5.79, min: 5.3, max: 6.24, range: 0.93 - Actual: 6.0
Predictions mean: 6.22, min: 5.98, max: 6.42, range: 0.44 - Actual: 7.0
Predictions mean: 6.24, min: 5.92, max: 6.42, range: 0.51 - Actual: 6.0


Train BNN with the whole training set.


In [13]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 43ms/step - loss: 39.8472 - root_mean_squared_error: 6.3121 - val_loss: 35.6552 - val_root_mean_squared_error: 5.9708
Epoch 2/500
17/17 [==============================] - 0s 7ms/step - loss: 37.2993 - root_mean_squared_error: 6.1070 - val_loss: 32.9806 - val_root_mean_squared_error: 5.7425
Epoch 3/500
17/17 [==============================] - 0s 7ms/step - loss: 33.8607 - root_mean_squared_error: 5.8186 - val_loss: 34.6920 - val_root_mean_squared_error: 5.8896
Epoch 4/500
17/17 [==============================] - 0s 7ms/step - loss: 33.5976 - root_mean_squared_error: 5.7960 - val_loss: 31.7498 - val_root_mean_squared_error: 5.6343
Epoch 5/500
17/17 [==============================] - 0s 7ms/step - loss: 31.4746 - root_mean_squared_error: 5.6098 - val_loss: 31.5175 - val_root_mean_squared_error: 5.6136
Epoch 6/500
17/17 [==============================] - 0s 7ms/step - loss: 30.3690 - root_mean_squared_error

Experiment 3: probabilistic Bayesian neural network

In [14]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 49ms/step - loss: 14.3700 - root_mean_squared_error: 5.6592 - val_loss: 10.7526 - val_root_mean_squared_error: 5.6228
Epoch 2/1000
17/17 [==============================] - 0s 7ms/step - loss: 10.4838 - root_mean_squared_error: 5.5792 - val_loss: 9.7981 - val_root_mean_squared_error: 5.7338
Epoch 3/1000
17/17 [==============================] - 0s 7ms/step - loss: 11.2194 - root_mean_squared_error: 5.5929 - val_loss: 13.1218 - val_root_mean_squared_error: 5.6511
Epoch 4/1000
17/17 [==============================] - 0s 7ms/step - loss: 8.9463 - root_mean_squared_error: 5.4140 - val_loss: 7.3806 - val_root_mean_squared_error: 5.3059
Epoch 5/1000
17/17 [==============================] - 0s 8ms/step - loss: 8.0995 - root_mean_squared_error: 5.4182 - val_loss: 6.6982 - val_root_mean_squared_error: 5.6007
Epoch 6/1000
17/17 [==============================] - 0s 7ms/step - loss: 8.2844 - root_mean_squared_error

In [16]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.17, stddev: 0.71, 95% CI: [6.55 - 3.79] - Actual: 5.0
Prediction mean: 6.12, stddev: 0.76, 95% CI: [7.6 - 4.63] - Actual: 6.0
Prediction mean: 5.08, stddev: 0.7, 95% CI: [6.45 - 3.7] - Actual: 5.0
Prediction mean: 6.22, stddev: 0.8, 95% CI: [7.78 - 4.66] - Actual: 6.0
Prediction mean: 5.75, stddev: 0.73, 95% CI: [7.19 - 4.31] - Actual: 6.0
Prediction mean: 5.06, stddev: 0.7, 95% CI: [6.43 - 3.68] - Actual: 5.0
Prediction mean: 5.97, stddev: 0.76, 95% CI: [7.45 - 4.48] - Actual: 7.0
Prediction mean: 5.87, stddev: 0.75, 95% CI: [7.33 - 4.4] - Actual: 6.0
Prediction mean: 6.23, stddev: 0.82, 95% CI: [7.84 - 4.62] - Actual: 7.0
Prediction mean: 6.27, stddev: 0.81, 95% CI: [7.86 - 4.68] - Actual: 6.0
